# Project Title: The Impact of News on the Market
### •	Team Members:
##### 	Rachel Torres, Christian Attard, Jess Alcalde, Nitin Khade
### •	Project Description/Outline:
##### -	We will look at news data and stock data to determine the effects of the news on how the market behaves.
### •	Research Questions to Answer:
##### -	How do news headlines affect the stock market?
##### -	Is there any correlation between certain types of headlines and effects on the market?
##### -	Does negative news affect stocks greater than positive or neutral?
##### -	Can we assign a factor(weighting) to it?

In [1]:
# import dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

from news_api import api_key
from x_api import x_api_key

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

# pip install vaderSentiment


### We'll query the news api to gather news headlines from the web

In [2]:
# parameters for news api
q='politics',
from_param='2019-05-24',
language='en',
sort_by='relevancy',
page = 4,
pageSize = 50

url = f'https://newsapi.org/v2/everything?q={q}&from={from_param}&language={language}&sortBy={sort_by}&pageSize={pageSize}&page={page}&apiKey={api_key}'
response = requests.get(url)

url_2 = f'https://newsapi.org/v2/everything?q=apple&from=2019-05-24&to=2019-06-23&language=en&sortBy=popularity&pageSize={pageSize}&page=3&apiKey={api_key}'
response = requests.get(url_2)
# Generate response into json 

In [3]:
pprint(response.json())

{'code': 'parameterInvalid',
 'message': 'You are trying to request results too far in the past. Your plan '
            'permits you to request articles as far back as 2019-05-25, but '
            'you have requested 2019-05-24. To extend this please upgrade to a '
            'paid plan.',
 'status': 'error'}


### We'll use a different API to gather news data since we have a restriction on the news api

In [4]:
# Query contextual web search API
# Replace the following string value with your valid X-RapidAPI-Key.
Your_X_RapidAPI_Key = x_api_key;

# The query parameters: (update according to your search query)
q = "Donald%20Trump%20News" #the search query
pageNumber = 1 #the number of requested page
pageSize = 50 #the size of a page
autoCorrect = True #autoCorrectspelling
safeSearch = False #filter results for adult content


# test response to query and get count of total items and pages
response_test=requests.get("https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI?q={}&pageNumber={}&pageSize={}&autocorrect={}&safeSearch={}".format(q, pageNumber, pageSize, autoCorrect,safeSearch),
headers={
"X-RapidAPI-Key": Your_X_RapidAPI_Key
}
).json()

#Get the number of items returned
totalCount = response_test["totalCount"];
totalPages = round(totalCount/pageSize)


In [5]:
pprint(response_test)

{'_type': 'news',
 'didUMean': '',
 'relatedSearch': ['president',
                   'iran',
                   'washington',
                   'white house',
                   'world <b>news</b>',
                   'fox',
                   'uk',
                   'views',
                   'abc',
                   'mexico',
                   'united states',
                   'took in as much'],
 'totalCount': 2425,
 'value': [{'datePublished': '2019-06-12T21:19:00',
            'description': 'President <b>Donald Trump</b> granted ABC '
                           '<b>News</b> an exclusive, wide-ranging interview '
                           'over the course of two days.',
            'image': {'base64Encoding': None,
                      'height': 558,
                      'thumbnail': 'https://contextualwebsearch.com/api/thumbnail/get?value=137322115715991574',
                      'thumbnailHeight': 139,
                      'thumbnailWidth': 247,
                    

In [6]:
# generate sample response to look at headers
pprint((response_test['value'][2]))

{'datePublished': '2019-06-26T18:03:52',
 'description': 'Rep. Adam Schiff, one of the two chairmen that subpoenaed '
                'Robert Mueller to testify, said Wednesday he would not '
                'characterize the demand as a friendly subpoena, and said the '
                'former special counsel is not eager to appear before Congress '
                'on July..  Politics  One <b>News</b> Page: Wednesday, 26 June '
                '2019',
 'image': {'base64Encoding': None,
           'height': 256,
           'thumbnail': 'https://contextualwebsearch.com/api/thumbnail/get?value=6187294181911490305',
           'thumbnailHeight': 256,
           'thumbnailWidth': 256,
           'url': 'https://cdn.newsserve.net/ONPglobe256.png',
           'width': 256},
 'isSafe': True,
 'keywords': 'president donald trump,business,news',
 'language': 'en',
 'provider': {'name': 'onenewspage'},
 'title': "Schiff: Mueller 'Not Eager' to Appear Before Congress",
 'url': 'https://www.onene

In [7]:
print(f' There are {totalPages} pages, with {totalCount} total articles.')

 There are 48 pages, with 2425 total articles.


In [8]:
# create empty lists to hold variable results
url_list = []
title_list = []
description_list = []
keywords_list = []
provider_list = []
date_list = []

# Load news data into pandas dataframe
# page = 1
for page in range(1,totalPages+1):
    response=requests.get("https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI?q={}&pageNumber={}&pageSize={}&autocorrect={}&safeSearch={}".format(q, page, pageSize, autoCorrect,safeSearch),
headers={"X-RapidAPI-Key": Your_X_RapidAPI_Key}).json()

# print(response)
    try:
        #Go over each resulting item
        for webPage in response["value"]:

#Get the web page metadata
            url = webPage["url"]
            title = webPage["title"]
            description = webPage["description"]
            keywords = webPage["keywords"]
            provider = webPage["provider"]["name"]
            datePublished = webPage["datePublished"]
        
        
            url_list.append(url)
            title_list.append(title)
            description_list.append(description)
            keywords_list.append(keywords)
            provider_list.append(provider)
            date_list.append(datePublished)

            print(f'printing {title}')    
    except (KeyError,IndexError):
            print('Not found, skipping')


printing ABC <b>News</b> travels with President <b>Donald Trump</b> from Iowa to the White House
printing <b>Donald Trump News</b>
printing Schiff: Mueller 'Not Eager' to Appear Before Congress
printing US Congress Pushing Back at <b>Trump</b> Over Saudi Arms Deals
printing Election 2020: Democratic debate preview
printing LIVE: Double blow to president as Mueller agrees to testify on Russia evidence and judge opens up his financial records
printing Foxs Sean Hannity,Tucker Carlson enjoy <b>Trump</b> rally ratings bonanza
printing Foxs Hannity, Carlson Enjoy <b>Trump</b> Rally Ratings Bonanza
printing Glendale man charged in theft of Marilyn Monroe statue pleads not guilty  Pasadena Star <b>News</b>
printing What you need to know about the US-Iran crisis, United States <b>News</b> & Top Stories
printing Fox's Hannity, Carlson enjoy <b>Trump</b> rally ratings bonanza
printing U.S. diplomacy festival won't put out Israel's fires
printing US-Iran crisis: <b>Trump</b> lashes out at 'ignora

printing United States committed to a U.K. trade deal after Brexit, says President <b>Donald Trump</b>
printing <b>Trump</b> says U.K. protests were small and fake <b>news</b>, but pictures tell a different story
printing <b>Donald Trump</b> says NHS will be part of post-Brexit trade deal negotiations
printing Hotstar drops trailer for Hindi version of The Office, gets hostile reception
printing <b>Donald Trump</b> protest held in Leamington
printing <b>Trump</b> says Brexit should and will happen during UK <b>news</b> conference
printing BBC blasted for hosting a <b>Donald Trump</b> baby blimp on BBC2 Victoria Derbyshire show  The Sun
printing Signatures Certified in Middleton School Board Recall Effort
printing Federal Judge Rejects Democrats Attempt to Block Border Wall Funding
printing <b>Trump</b> transition team associate arrested in NY on child porn charges
printing <b>Donald Trump</b> is angry because he cant watch Fox <b>News</b> in the UK
printing The Ron Paul Institute for P

printing Amid tough talk, <b>Trump</b> says he could be Iran's 'best friend'
printing Im going to be Irans best friend: <b>Donald Trump</b>
printing Former <b>Trump</b> aide agrees to judiciary interview
printing North Korea Leader Receives Excellent Letter From <b>Trump</b>
printing Amid Tough Talk, <b>Trump</b> Says He Could Be Irans Best Friend
printing <b>TRUMP</b> CALLS OFF ICE RAIDS: Postpones Nationwide Immigration Enforcement Sweep
printing <b>Trump</b> to impose 'major' new sanctions on Iran on Monday
printing <b>Donald Trump</b> delays ICE's mass deportations of families as he seeks deal with Democrats
printing Palestinians reject Kushner's $50b bid to 'buy' Mideast peace
printing <b>Donald Trump</b> softens stance against Iran after US drone shot down
printing After calling off strikes on Iran, <b>Trump</b> suggests patience
printing North Korea's Kim Jong Un receives personal letter from <b>Trump</b>, KCNA says, East Asia <b>News</b> & Top Stories
printing By calling off st

printing How <b>Donald Trump's</b> Russia blind spot is hamstringing his administration
printing [<b>News</b> Focus] <b>Trump</b> to launch 2020 campaign in retiree-friendly Florida
printing <b>Trump</b> now a poster boy for the far-right, says London Mayor
printing Wife of former Nissan boss Carlos Ghosn seeks US President <b>Donald Trumps</b> help
printing London mayor Sadiq Khan attacks police cuts after four murders in four days
printing <b>Trump</b> assumes financial statement' will be released at some point'
printing <b>Donald Trump</b> stirs up feud with London Mayor
printing <b>Trump</b> campaign parting ways with pollsters following internal numbers leak: Sources
printing Israeli PM names village after <b>Donald Trump</b> as thanks for Golan Heights support
printing <b>Donald Trump</b> marks four years since starting his journey to the Oval Office
printing Benjamin Netanyahu Names Illegal Israeli Settlement <b>Trump</b> Heights After <b>Donald Trump</b>
printing <b>Trump</b> s

printing China mum on Trump-Xi meeting as Huawei says sales slowing
printing Biden assailing <b>Trump</b> on trade as both come to Iowa  WSVN 7News
printing <b>Trump</b>, Biden head to Iowa in potential 2020 preview
printing House Resolution Would Make It Easier To Enforce Subpoenas
printing <b>Trump</b> says Fed's independence hurts him in trade war with China
printing Mexican tariffs bad, Chinese tariffs good? <b>Trumps</b> trade war lifts 'factory North America'
printing <b>Trump news</b>  live: 'First-hand accounts' of president's obstruction to be revealed after Justice Department deal over Mueller evidence
printing New York Times to cease political cartoons after anti-Semitism row
printing <b>Trump</b> ready to slap more tariffs on China after G20 meeting
printing <b>Donald Trump</b> ready to slap more tariffs on China after G20 meeting
printing Asian shares rise as <b>Trump</b> plans to meet Chinese leader
printing Mexico denies <b>Trump's</b> claim of secret concessions in deal

...

printing <b>Trump</b> Won't Rush North Korea on Denuclearization; Peace Deal Possible
printing <b>Donald Trump</b> delays tariff hike on Chinese goods citing trade talk progress
printing North Korea leader Kim Jong-un boards train to summit
printing North Korean leader's train arrives in China: Reports
printing India and Pakistan in a ‘very delicate balance’, says <b>Donald Trump</b>
printing Democratic Party Adopts Motion Calling on US to Re-Enter Iran Nuclear Deal
printing <b>Donald Trump</b> refuses to allow US-born woman who joined ISIS to return
printing Mueller’s probe into Russian meddling in 2016 US election is ending: Reports
printing <b>Trump</b> administration wants California to pay back $2.5 billion for bullet train
printing <b>Donald Trump</b> describes Pulwama terror attack as ‘horrible’, urges Pakistan to ‘fully cooperate’
printing Bernie Sanders joins 2020 race, to defeat ‘racist’ <b>Donald Trump</b>
printing Mexico border wall: US states sue over emergency declaration

printing Turkey welcomes US plan for Syria ‘safe zone’
printing Shutdown meal? White House hosts college football champs with fast food dinner
printing <b>Trump</b>: I'll never back down
printing Anti-Trump media amp the outrage and fuel public suspicion
printing <b>Trump's</b> call-in to Fox <b>News</b> backfires, and he may have committed a felony in the process
printing Pompeos Words Are Irans Worst Nightmare
printing <b>Trump</b>: Obama took Biden off the 'trash heap'
printing <b>Trump</b> calls Joe Biden 'weak,' says Obama took him off the 'trash heap'
printing As US govt shutdown enters day 22, <b>Donald Trump</b> says can get deal done in 15 min
printing <b>Trump</b> says no emergency declaration to end US govt shutdown » Capital <b>News</b>
printing <b>Donald Trump</b> takes on FBI for probe against him into Russia links
printing <b>Trump</b>: No emergency declaration to end US govt shutdown
printing A few words of comfort for a confused region
printing Gov.s husband sending <b

printing City Jewish body demands <b>Trump</b> rejects nationalism
printing Sarah Sanders can't name a single 'fake <b>news</b>' outlet
printing <b>Trump's</b> phony 'caravan' hysteria led to the Synagogue murders: Adam Serwer for The Atlantic
printing <b>Trump</b> Says Pittsburgh Shooting Has Little to Do With Gun Laws
printing <b>Trump</b> hints he could curb immigration by decree » Capital <b>News</b>
printing Chilling mail-bomb plot
printing Khashoggi fiancée snubs <b>Trump</b> invitation
printing Seeking 'Unprecedented' Pressure on Iran, <b>Trump</b> Reimposes US Sanctions Lifted After Nuclear Deal
printing White House invites Vladimir Putin to Washington
printing 5 things to know about <b>Trump's</b> war (of words) on the Fed
printing Bombs sent to <b>Trump</b> critics Biden, De Niro as manhunt intensifies
printing US probes pipe bombs sent to <b>Trump</b> foes Obama, Clinton, CNN
printing <b>Trump</b> Says Saudis Staged 'Worst Cover-Up Ever' on Khashoggi
printing 2018 election: 

printing <b>Trump</b> pardons ranchers
printing <b>Trump</b> picks Kavanaugh
printing China and Russia hit back at <b>Trump</b> tariffs
printing <b>Trump</b> to meet May, Queen and visit Churchill's birthplace » Capital <b>News</b>
printing Under Fire for Ethics Scandals, EPA Chief Pruitt Resigns
printing 'American heroes' hailed
printing <b>Trump</b> vets judges
printing <b>Trump's</b> gunboat diplomacy
printing <b>Trump</b> interviews for justice
printing Oprah not running for prez
printing Mexico's new president
printing Coast to Coast protests
printing <b>Trump</b> blackmail: Mexico
printing Bias against <b>Trump</b> in FBI?
printing <b>Trump's</b> press secretary kicked out of restaurant
printing <b>Trump</b> tries to change focus
printing <b>Trump</b> sends letter to Justin
printing <b>Trump</b> the peacemaker?
printing <b>Trump</b> ends heartbreak
printing Kids will be reunited
printing Order could reunite families
printing Unapologetic <b>Trump</b> digs in
printing Policy divid

printing Obama trade chief says TPP may still pass Congress after elections – report  — RT USA <b>News</b>
printing Democrats sue <b>Trump</b> & GOP, claiming minority voters intimidation under Klan law  RT USA <b>News</b>
printing <b>Trump</b> warns US presidential election will be ‘rigged’, calls Hillary Clinton ‘the Devil’  — RT USA <b>News</b>
printing Online outrage over NY Posts nude pics of Melania <b>Trump</b>, <b>Donald</b> not too bothered  RT USA <b>News</b>
printing <b>Trump</b> campaign manager arrested for assaulting Breitbart reporter  RT USA <b>News</b>
printing Sophisticated, political maneuvering in Louisiana by Cruz could cost <b>Trump</b> — RT USA <b>News</b>
printing <b>Trump</b> defiantly speaks after protest blockade delays Arizona rally — RT USA <b>News</b>
printing Best moments from the GOP debate  RT USA <b>News</b>
printing Secret Service promises ‘appropriate steps’ after investigating agent’s assault on TIME photographer — RT USA <b>News</b>
printing <b>Don

In [9]:
# assign list to dataframe

newsFrame = pd.DataFrame({'datePublished':date_list,'description':description_list,'keywords':keywords_list,'provider':provider_list,'title':title_list,'url':url_list})


In [10]:
newsFrame.head(200)

,datePublished,description,keywords,provider,title,url
0,2019-06-12T21:19:00,President <b>Donald Trump</b> granted ABC <b>N...,"president donald trump,donald trump news,white...",go,ABC <b>News</b> travels with President <b>Dona...,https://preview.abcnews.go.com/politics/abc-ne...
1,2019-05-22T23:00:00,<b>Donald Trump News</b> from United Press Int...,"donald trump news,photos,quotes,press",upi,<b>Donald Trump News</b>,https://www.upi.com/topic/donald_trump/?tps=1/...
2,2019-06-26T18:03:52,"Rep. Adam Schiff, one of the two chairmen that...","president donald trump,business,news",onenewspage,Schiff: Mueller 'Not Eager' to Appear Before C...,https://www.onenewspage.com/n/Politics/1zkidwp...
3,2019-06-26T14:36:12,US President <b>Donald Trump</b> holds a chart...,"president donald trump,israel news",algemeiner,US Congress Pushing Back at <b>Trump</b> Over ...,https://www.algemeiner.com/2019/06/26/us-congr...
4,2019-06-26T12:00:00,With the Illinois primary still another 265 da...,"donald trump,democrats,miami,news",news-gazette,Election 2020: Democratic debate preview,http://www.news-gazette.com/news/local/2019-06...
5,2019-06-26T07:07:00,<b>Donald</b> Trumphas received two fresh setb...,"donald trump,trump news,president,house",independent,LIVE: Double blow to president as Mueller agre...,https://www.independent.co.uk/news/world/ameri...
6,2019-06-26T03:50:00,LOS ANGELES >> Ratings figures show Fox <b>New...,"president donald trump,fox news",staradvertiser,"Foxs Sean Hannity,Tucker Carlson enjoy <b>Trum...",https://www.staradvertiser.com/2019/06/25/news...
7,2019-06-26T02:00:32,Ratings figures show Fox <b>News</b> hosts Sea...,"president donald trump,news",theyeshivaworld,"Foxs Hannity, Carlson Enjoy <b>Trump</b> Rally...",https://www.theyeshivaworld.com/news/general/1...
8,2019-06-26T02:00:07,"Suspect Austin Mikel Clay, 25, is the same man...","president donald trump,news,man",pasadenastarnews,Glendale man charged in theft of Marilyn Monro...,https://www.pasadenastarnews.com/2019/06/25/gl...
9,2019-06-26T00:53:13,WASHINGTON - US President <b>Donald Trump</b> ...,"president donald trump,iran,news",straitstimes,What you need to know about the US-Iran crisis...,http://www.straitstimes.com/world/united-state...


In [11]:
newsFrame.dtypes

datePublished    object
description      object
keywords         object
provider         object
title            object
url              object
dtype: object

In [12]:
# clean description and title columns
newsFrame.description = newsFrame.description.str.replace('<b>','')
newsFrame.description = newsFrame.description.str.replace('</b>','')
newsFrame.title = newsFrame.title.str.replace('<b>','')
newsFrame.title = newsFrame.title.str.replace('</b>','')

In [13]:
newsFrame.head()

,datePublished,description,keywords,provider,title,url
0,2019-06-12T21:19:00,President Donald Trump granted ABC News an exc...,"president donald trump,donald trump news,white...",go,ABC News travels with President Donald Trump f...,https://preview.abcnews.go.com/politics/abc-ne...
1,2019-05-22T23:00:00,Donald Trump News from United Press Internatio...,"donald trump news,photos,quotes,press",upi,Donald Trump News,https://www.upi.com/topic/donald_trump/?tps=1/...
2,2019-06-26T18:03:52,"Rep. Adam Schiff, one of the two chairmen that...","president donald trump,business,news",onenewspage,Schiff: Mueller 'Not Eager' to Appear Before C...,https://www.onenewspage.com/n/Politics/1zkidwp...
3,2019-06-26T14:36:12,US President Donald Trump holds a chart of mil...,"president donald trump,israel news",algemeiner,US Congress Pushing Back at Trump Over Saudi A...,https://www.algemeiner.com/2019/06/26/us-congr...
4,2019-06-26T12:00:00,With the Illinois primary still another 265 da...,"donald trump,democrats,miami,news",news-gazette,Election 2020: Democratic debate preview,http://www.news-gazette.com/news/local/2019-06...


In [14]:
newsFrame.count()

datePublished    2010
description      2010
keywords         2010
provider         2010
title            2010
url              2010
dtype: int64

In [15]:
# Query sample news description
newsFrame.iloc[7][1]

'Ratings figures show Fox News hosts Sean Hannity and Tucker Carlson enjoyed a ratings surge from last weeks 2020 Trump campaign kickoff rally. Viewers watching last weeks event in Orlando, Florid'

In [ ]:
# import stock data 

# Load stock data into notebook as dataframe

In [ ]:
# Merge 2 DataFrames

### About the Scoring (taken from vaderSentiment docs)
The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:

    positive sentiment: compound score >= 0.05
    neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
    negative sentiment: compound score <= -0.05
The pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all add up to be 1... or close to it with float operation). These are the most useful metrics if you want multidimensional measures of sentiment for a given sentence.

In [32]:
# Do sentiment analysis of news data

# Define function to test 
def sentiment_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} \n \n{}".format(sentence, str(score)))
    
# Do test of single string
sentiment_scores(newsFrame.iloc[10][1])

LOS ANGELES  Fox News hosts Sean Hannity and Tucker Carlson enjoyed a ratings surge from the channels solo extended coverage of President Donald Trumps 2020 campaign kickoff. Viewers watching last weeks rally in Orlando, Florida, helped cable star Hannity earn a rare top 10 showing among both broadcast and cable programs with 5 million 
 
{'neg': 0.0, 'neu': 0.906, 'pos': 0.094, 'compound': 0.6249}


In [ ]:
# Use Matplotlib and stats to generate graphs and look for relationships